In [27]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sqlalchemy import create_engine

In [28]:
pd.options.plotting.backend = "plotly"

In [29]:
creds = {'DB_HOST': 'live-main.c56d9jndqmv2.us-east-2.redshift.amazonaws.com',
         'DB_PORT': '5439', 
         'DB_NAME': 'klarprod',
         'DB_USER': 'tf_juan_sarmiento_an',
         'DB_PASSWORD': 'nVTDW96Ag', 
         }

conn_normal = 'postgresql+psycopg2://' + creds['DB_USER'] + ':' + creds['DB_PASSWORD'] + '@' + creds['DB_HOST'] + ':' + creds['DB_PORT'] + '/' + creds['DB_NAME']
#cnx = create_engine(conn_normal)

In [30]:
# Redshift connection
f = open('/Users/gabrielreynoso/Documents/Queries/db_klarprod_connection.txt', 'r')
postgres_str = f.read()
f.close()
cnx = create_engine(postgres_str)

In [31]:
query = '''
select *
from unit_economics.debit_monthly_economics
'''

In [32]:
debit_df_whole = pd.read_sql_query(query, cnx)

In [33]:
#censoring
debit_df = debit_df_whole[debit_df_whole['debit_business_month'] >= 0]
debit_df = debit_df[debit_df['transaction_active_customers'] > 1]
debit_df['actual_month'] = debit_df.apply(lambda x: x.debit_monthly_cohort + pd.DateOffset(months=x.debit_business_month, days=0), axis=1)

In [34]:
profit_loss = debit_df.groupby(by='actual_month').aggregate(
    {'contribution_profit_30': 'sum',
     'transaction_active_customers': 'sum',
     'gross_revenue_30': 'sum',
     'gross_npl_30': 'sum'}).reset_index()
profit_loss['contribution_margin'] = (profit_loss['contribution_profit_30']) / profit_loss['transaction_active_customers']
profit_loss['revenues_usd'] = profit_loss.gross_revenue_30 / 20
profit_loss['losses_usd'] = profit_loss.gross_npl_30 / 20
profit_loss = profit_loss.style.format(formatter={'revenues_usd': '{:,.2f}', 'losses_usd': '{:,.2f}'})

In [35]:
profit_loss

,actual_month,contribution_profit_30,transaction_active_customers,gross_revenue_30,gross_npl_30,contribution_margin,revenues_usd,losses_usd
0,2020-07-01 00:00:00,55924.631036,9540,334640.851036,278716.220000,5.862121,"16,732.04","13,935.81"
1,2020-08-01 00:00:00,20825.485232,21640,410914.425232,390088.940000,0.962361,"20,545.72","19,504.45"
2,2020-09-01 00:00:00,-273262.321367,19405,674478.698633,947741.020000,-14.082057,"33,723.93","47,387.05"
3,2020-10-01 00:00:00,323577.934504,27511,786443.514504,462865.580000,11.761766,"39,322.18","23,143.28"
4,2020-11-01 00:00:00,671773.984057,33441,912877.454057,241103.470000,20.088334,"45,643.87","12,055.17"
5,2020-12-01 00:00:00,-286279.797017,38489,1687468.952983,1973748.750000,-7.437964,"84,373.45","98,687.44"
6,2021-01-01 00:00:00,-813572.480998,46150,2040210.549002,2853783.030000,-17.628873,"102,010.53","142,689.15"
7,2021-02-01 00:00:00,-1619331.159992,55753,3335757.400008,4955088.560000,-29.044736,"166,787.87","247,754.43"
8,2021-03-01 00:00:00,-5857725.873816,76350,6579206.176184,12436932.050000,-76.722015,"328,960.31","621,846.60"
9,2021-04-01 00:00:00,-7538903.045066,96696,8471582.744934,16010485.790000,-77.964994,"423,579.14","800,524.29"


In [36]:
#full-censoring
debit_df = debit_df[debit_df['full_loan_censoring_30'] == 1]

In [37]:
profit_loss = debit_df.groupby(by='actual_month').aggregate({'contribution_profit_30': 'sum', 'transaction_active_customers': 'sum', 'gross_revenue_30': 'sum', 'gross_npl_30': 'sum'}).reset_index()
profit_loss['contribution_margin'] = (profit_loss['contribution_profit_30']) / profit_loss['transaction_active_customers']
profit_loss['revenues_usd'] = profit_loss.gross_revenue_30 / 20
profit_loss['losses_usd'] = profit_loss.gross_npl_30 / 20
profit_loss_styled = profit_loss.style.format(formatter={'revenues_usd': '{:,.2f}', 'losses_usd': '{:,.2f}'})

In [38]:
profit_loss_styled

,actual_month,contribution_profit_30,transaction_active_customers,gross_revenue_30,gross_npl_30,contribution_margin,revenues_usd,losses_usd
0,2020-07-01 00:00:00,55924.631036,9540,334640.851036,278716.220000,5.862121,"16,732.04","13,935.81"
1,2020-08-01 00:00:00,20825.485232,21640,410914.425232,390088.940000,0.962361,"20,545.72","19,504.45"
2,2020-09-01 00:00:00,-273262.321367,19405,674478.698633,947741.020000,-14.082057,"33,723.93","47,387.05"
3,2020-10-01 00:00:00,323577.934504,27511,786443.514504,462865.580000,11.761766,"39,322.18","23,143.28"
4,2020-11-01 00:00:00,671773.984057,33441,912877.454057,241103.470000,20.088334,"45,643.87","12,055.17"
5,2020-12-01 00:00:00,-286279.797017,38489,1687468.952983,1973748.750000,-7.437964,"84,373.45","98,687.44"
6,2021-01-01 00:00:00,-813572.480998,46150,2040210.549002,2853783.030000,-17.628873,"102,010.53","142,689.15"
7,2021-02-01 00:00:00,-1619331.159992,55753,3335757.400008,4955088.560000,-29.044736,"166,787.87","247,754.43"
8,2021-03-01 00:00:00,-5857725.873816,76350,6579206.176184,12436932.050000,-76.722015,"328,960.31","621,846.60"
9,2021-04-01 00:00:00,-7538903.045066,96696,8471582.744934,16010485.790000,-77.964994,"423,579.14","800,524.29"


In [39]:
profit_loss.to_clipboard()